In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Twitter_Data.csv


In [2]:
! pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.1 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, Dataset, DatasetDict
# import pandas as pd

# df = pd.read_csv("/kaggle/input/twitter-sentiment-dataset/Twitter_Data.csv")

from datasets import load_dataset

dataset = load_dataset("carblacac/twitter-sentiment-analysis")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-992cdfca7a4ca469/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/149985 [00:00<?, ?ex/s]

...

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

Dataset new_dataset downloaded and prepared to /root/.cache/huggingface/datasets/carblacac___new_dataset/default/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# df = df.rename(columns={"clean_text":"text", "category":"label"})

In [6]:
# df=df.dropna()

In [76]:
# df["label"] = df["label"].replace(0, 1)
# df["label"] = df["label"].replace(-1, 0)

In [77]:
# columns_titles = ["label","text"]
# df=df.reindex(columns=columns_titles)
# df["label"] = df["label"].astype(int)

In [78]:
# df

,label,text
0,0,when modi promised “minimum government maximum...
1,1,talk all the nonsense and continue all the dra...
2,1,what did just say vote for modi welcome bjp t...
3,1,asking his supporters prefix chowkidar their n...
4,1,answer who among these the most powerful world...
...,...,...
162975,0,why these 456 crores paid neerav modi not reco...
162976,0,dear rss terrorist payal gawar what about modi...
162977,1,did you cover her interaction forum where she ...
162978,1,there big project came into india modi dream p...


In [79]:
# from sklearn.model_selection import train_test_split

# # Split the dataset into train and test sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=1)

# # Further split the train set into train and validation sets
# test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=1)

# train_dataset = Dataset.from_pandas(train_df).remove_columns("__index_level_0__")
# test_dataset = Dataset.from_pandas(test_df).remove_columns("__index_level_0__")
# val_dataset = Dataset.from_pandas(val_df).remove_columns("__index_level_0__")

# datasets = DatasetDict({
#     'train': train_dataset,
#     'validation': val_dataset,
#     'test': test_dataset
# })

In [30]:
datasets = dataset.rename_column("feeling", "label")

In [31]:
datasets["test"][0]

{'text': "@justineville ...yeahhh. ) i'm 39 tweets from 1,600!", 'label': 1}

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("windshield-viper/discord-distilbert")

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [11]:
def preprocess_function(examples):

    return tokenizer(examples["text"], truncation=True)

In [41]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

In [42]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 867.6 kB/s eta 0:00:000:00:010m


In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [43]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)



In [44]:
id2label = {0: "NEGATIVE", 1:"POSITIVE"}

label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [45]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "windshield-viper/discord-distilbert", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at windshield-viper/discord-distilbert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)



In [47]:
import torch

In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()

In [94]:
tokenizer.push_to_hub("windshield-viper/my_awesome_model")

README.md:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/windshield-viper/my_awesome_model/commit/f362753403bb3c84115b4479bbde0db5f17053e9', commit_message='Upload tokenizer', commit_description='', oid='f362753403bb3c84115b4479bbde0db5f17053e9', pr_url=None, pr_revision=None, pr_num=None)